In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -U sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 75.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 83.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 24.3 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=ed95af1b88d32587b28dc27cd1bfd47bf9a5eb8b8c06ea472565bf85cd9cc413
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [3]:
from transformers import AutoTokenizer, AutoModel
import torch
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from sentence_transformers import SentenceTransformer, util, evaluation
import random
import json
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device "{device}"')

Using device "cuda"


In [17]:
class parser():
  def __init__(self):

    self.data_dir= "/content/drive/MyDrive/pRoBERTa/data/converted_data_train_tocken.json"
    self.save_model_path="/content/drive/MyDrive/pRoBERTa/trained_module/rationale_selection/huggingface_tocken_freeze"
    self.save_checkpoint = "/content/drive/MyDrive/pRoBERTa/trained_module/rationale_selection/huggingface_tocken_freeze/checkpoint"
    self.scale_pos_neg = 10

args = parser()

In [6]:
model = SentenceTransformer('keepitreal/vietnamese-sbert')

# # Path to your trained model's pytorch_model.bin file
# model_path = '/content/drive/MyDrive/pRoBERTa/trained_module/rationale_selection/huggingface_tocken/'

# # Load the model
# model = SentenceTransformer(model_path)


In [7]:
sentences = ["ở vòng này độc giả vnexpress sẽ bình chọn cho sản phẩm / giải pháp mà mình yêu thích và đánh giá cao", "vì vậy sau khi trở lại sài gòn chị đã quyết định mở hàng ốc nhồi hơn một năm trước"]
embeddings = model.encode(sentences)
print(embeddings)
util.pytorch_cos_sim(embeddings[0], embeddings[1])

[[-0.09916721  0.28454182 -0.12842584 ...  0.3223491   0.14667584
  -0.11820575]
 [-0.01042098 -0.03861305  0.18633303 ...  0.33944547 -0.18970798
  -0.19059181]]


tensor([[0.1355]])

# Chuẩn bị dữ liệu

In [20]:
import random
from torch.utils.data import Dataset
import json
import pandas as pd
class LoadDataFromJson(Dataset, ):
    def __init__(self, datasets):
        self.samples = []
        with open(datasets) as file:
            data = json.load(file)

        for id, item in data.items():
            sentences = item["context"]
            if item["evidence"] != None:
              for sentence in sentences:
                data_sample = {}
                evidence = item['evidence'] in sentence or sentence in item['evidence']
                if evidence:
                  data_sample['evidence_label'] = 1
                else:
                  data_sample['evidence_label'] = 0
                data_sample['id'] = id
                data_sample['claim'] = item['claim']
                data_sample['sentence'] = sentence
                data_sample['evidence'] = item['evidence']

                data_sample["verdict"] = item["verdict"]
                data_sample['domain'] = item['domain']
                self.samples.append(data_sample)

    def to_dataframe(self):
        return pd.DataFrame(self.samples).sample(frac=1, random_state=3)

In [21]:
dat_path = args.data_dir
a = LoadDataFromJson(dat_path)
df = a.to_dataframe()
df

,evidence_label,id,claim,sentence,evidence,verdict,domain
234309,0,30927,khi gia_nhập college compass nhung được kết_nố...,trong bài_luận gửi đến trường harvard nữ_sinh_...,khi trở_thành học_sinh của college compass nhu...,SUPPORTED,giaoduc
223745,0,30428,từ lúc nghỉ_việc gia_đình chị thúy rất khó khắ...,lê tuyết,từ lúc nghỉ_việc gia_đình chị thúy chỉ kiếm đư...,SUPPORTED,thoisu
94193,0,11558,cần nhiều công_nghệ và thời_gian để khắc_phục ...,khi tìm_kiếm sản_phẩm chăm_sóc da đặc_biệt cho...,bác_sĩ thành cho rằng trường_hợp bệnh_nhân trê...,SUPPORTED,suckhoe
114060,0,12370,ông trump không_thể tự ân_xá cho mình dù trở_t...,quá_trình xử_lý vụ án có_thể diễn ra trong thờ...,nếu trở_thành tổng_thống ông trump không nắm g...,SUPPORTED,thegioi
193742,0,19637,phòng lao_động quận 7 ban quản_lý các khu chế_...,khảo_sát của công_đoàn cho thấy khoảng 240 lao...,hơn tháng qua công_đoàn đã trực_tiếp liên_hệ v...,SUPPORTED,thoisu
...,...,...,...,...,...,...,...
11261,0,7631,tháng 4/2020 là thời_kỳ đỉnh_điểm nhất của cov...,đặc_biệt sản_phẩm này có tiềm_năng hỗ_trợ ngườ...,giai_đoạn cao_điểm covid19 hồi tháng 4 năm 202...,SUPPORTED,khoahoc
48056,0,9398,vật_liệu điện_sắc được giới khoa_học quan_tâm ...,thiết_bị điện_sắc được nhóm nghiên_cứu thử_ngh...,theo nhóm vật_liệu điện_sắc được giới khoa_học...,SUPPORTED,khoahoc
77049,0,10830,những chiến_thắng được ca_ngợi giúp prigozhin ...,các thành_viên wagner được cho là đã âm_thầm t...,điều đó cũng giúp prigozhin tăng_cường ảnh_hưở...,SUPPORTED,thegioi
198296,0,19911,những bức ảnh được nhiếp_ảnh gia_john thomson ...,áo_dài năm thân_may theo phong_cách kín_đáo đư...,thiếu_nữ sài_gòn là một trong những bức ảnh đư...,SUPPORTED,thegioi


In [22]:
df["evidence_label"].value_counts()

0    322111
1     14550
Name: evidence_label, dtype: int64

In [23]:
filtered_df = df.loc[(df['sentence'].apply(lambda x: len(x.split())) < 4)]
filtered_df

,evidence_label,id,claim,sentence,evidence,verdict,domain
223745,0,30428,từ lúc nghỉ_việc gia_đình chị thúy rất khó khắ...,lê tuyết,từ lúc nghỉ_việc gia_đình chị thúy chỉ kiếm đư...,SUPPORTED,thoisu
54650,0,9735,giảo cổ_lam,thanh hoa,giảo cổ_lam trong giảo cổ_lam có chứa phanosid,SUPPORTED,suckhoe
162702,0,14185,đầu năm 2022 dự_án sẽ đi vào hoạt_động,để phòng nguy_hiểm,đầu năm 2022 công_trình đi vào hoạt_động,SUPPORTED,thoisu
160332,0,14075,rạng sáng nay người_dân khu_vực gần cầu kênh n...,ngọc tài nguyễn_khánh,rạng sáng nay người_dân gần cầu bắc qua kênh n...,SUPPORTED,thoisu
214061,0,30071,trong năm đầu_tiên cô đã hoàn_thành_phần nghiê...,thanh hằng,năm đầu_tiên cô hoàn_thành_phần nghiên_cứu lý_...,SUPPORTED,giaoduc
...,...,...,...,...,...,...,...
227131,0,30588,trong thông_báo vừa gửi 7 hãng taxi về giá dịc...,gia_minh,trong thông_báo vừa gửi 7 hãng taxi về giá dịc...,SUPPORTED,thoisu
198769,0,19936,yoga xuất_hiện từ thời ấn_độ cổ_đại,bảo đức,yoga là bộ_môn truyền_thống năm nghìn năm_tuổi...,SUPPORTED,thegioi
185884,1,19243,dự_kiến tổng mức đầu_tư cho dự_án cảng_hàng_kh...,phước tuấn_gia chính,phước tuấn_gia chính,SUPPORTED,thoisu
306027,0,46364,ông nguyễn_quang_thái thành_viên hội_đồng tư_v...,hà_an,ông nguyễn_quang_thái thành_viên hội_đồng tư_v...,REFUTED,khoahoc


In [24]:
df = df.drop(df[df.isin(filtered_df.to_dict('list')).all(1)].index)

In [25]:
df_eval_1 = df[df['evidence_label'] == 1].head(690)
df_eval_0 = df[df['evidence_label'] == 0].head(690)
df_eval = pd.concat([df_eval_0, df_eval_1])


In [26]:
df_train = df[~df.isin(df_eval).all(axis=1)]

In [27]:
df_train_1 = df_train[df_train['evidence_label'] == 1]
df_train_0 = df_train[df_train['evidence_label'] == 0].head(len(df_train_1)*args.scale_pos_neg)
df_train = pd.concat([df_train_0, df_train_1])

In [28]:
df_train['evidence_label'].value_counts()


0    132560
1     13256
Name: evidence_label, dtype: int64

In [29]:
from sentence_transformers import SentenceTransformer, LoggingHandler, losses, InputExample
batch_size = 16
sampler = torch.utils.data.sampler.RandomSampler
# batch_sampler = torch.utils.data.sampler.BatchSampler

train_examples = []
for index, row in df_train.iterrows():
    claim = row['claim']
    sentence = row['sentence']
    evidence_label = row['evidence_label']
    input = InputExample(texts=[claim, sentence], label=evidence_label)
    train_examples.append(input)

train_dataloader = DataLoader(train_examples, batch_size=batch_size)

eval_examples = []
for index, row in df_eval.iterrows():
    claim = row['claim']
    sentence = row['sentence']
    evidence_label = row['evidence_label']
    input = InputExample(texts=[claim, sentence], label=evidence_label)
    eval_examples.append(input)


# Train

In [30]:
model = SentenceTransformer('keepitreal/vietnamese-sbert')

In [31]:

from sentence_transformers import SentenceTransformer, LoggingHandler, losses, InputExample
from torch.utils.data import DataLoader

# # Path to your trained model's pytorch_model.bin file
# model_path = '/content/drive/MyDrive/pRoBERTa/trained_module/rationale_selection/huggingface_tocken/'
# # Load the model
# model = SentenceTransformer(model_path)

evaluator = evaluation.BinaryClassificationEvaluator.from_input_examples(eval_examples)

train_loss = losses.ContrastiveLoss(model=model)


model.fit([(train_dataloader, train_loss)],
          show_progress_bar = True,
          epochs=10,
          evaluation_steps = 1000,
          evaluator = evaluator,
          max_grad_norm = 1,
          optimizer_class = torch.optim.AdamW,
          optimizer_params = {"lr" : 2e-05},
          scheduler = "WarmupLinear",
          steps_per_epoch = None,
          warmup_steps = 144,
          weight_decay = 0.01,
          # checkpoint_path = args.save_checkpoint,
          output_path = args.save_model_path
          )


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/9114 [00:00<?, ?it/s]

Iteration:   0%|          | 0/9114 [00:00<?, ?it/s]

Iteration:   0%|          | 0/9114 [00:00<?, ?it/s]

Iteration:   0%|          | 0/9114 [00:00<?, ?it/s]

Iteration:   0%|          | 0/9114 [00:00<?, ?it/s]

Iteration:   0%|          | 0/9114 [00:00<?, ?it/s]

Iteration:   0%|          | 0/9114 [00:00<?, ?it/s]

KeyboardInterrupt: ignored